In [ ]:
# download the data
# !wget -N https://amldata.s3.eu-central-1.amazonaws.com/data.zip
# !unzip -o data.zip

In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_hub as hub
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.models import Sequential
from keras.metrics import sparse_top_k_categorical_accuracy
from sklearn.preprocessing import normalize

In [ ]:
# immage dimensions
train_path = os.getcwd() + '/data/train_set/'+'train/'
img_height = 224
img_width = 224
IMAGE_SHAPE = (img_height, img_width)

In [ ]:
#init training and val data
batch_size = 32
seed = 16

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    labels='inferred', 
    label_mode='int',
    color_mode='rgb',
)
  

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    labels='inferred', 
    label_mode='int',
    color_mode='rgb',
)

In [ ]:
class_names = train_ds.class_names
nr_labels = len(class_names)
print('lables:', class_names)

In [ ]:
# plot images
plt.figure(figsize=(20, 20))
for images, labels in train_ds.take(1):
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# fetch pre trained models
res_net = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE+(3,),
                                               include_top=False,
                                               weights='imagenet',
                                           )
mobile_net = tf.keras.applications.ResNet50V2(input_shape=IMAGE_SHAPE+(3,),
                                               include_top=False,
                                               weights='imagenet',
                                             )
nas_net = tf.keras.applications.NASNetMobile(input_shape=IMAGE_SHAPE+(3,),
                                               include_top=False,
                                               weights='imagenet',
                                             )

mobile_net.trainable = False
res_net.trainable = False
nas_net.trainable = False


In [ ]:
# variables for the models
scale = 1./127.5
offset = -1
zoom_per = 0.2

In [ ]:
# create models


ResNet = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(scale, offset=offset),
    layers.experimental.preprocessing.RandomZoom(zoom_per),
    res_net,
    layers.GlobalAveragePooling2D(),
    layers.Dense(nr_labels),
])

MobileNet = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(scale, offset=offset),
    layers.experimental.preprocessing.RandomZoom(zoom_per),
    mobile_net,
    layers.GlobalAveragePooling2D(),
    layers.Dense(nr_labels),
])
NASNet = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(scale, offset=offset),
    layers.experimental.preprocessing.RandomZoom(zoom_per),
    nas_net,
    layers.GlobalAveragePooling2D(),
    layers.Dense(nr_labels),
])



In [ ]:
# compile models

ResNet.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

MobileNet.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


NASNet.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
epochs = 10

In [ ]:
checkpoint_path = "ResNet/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

ResNet.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    shuffle=True,
    callbacks=[cp_callback]
)

In [ ]:
!mkdir -p final
ResNet.save('final/ResNet.h5')

In [ ]:
checkpoint_path = "MobileNet/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
MobileNet.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    shuffle=True,
    callbacks=[cp_callback]
)

In [ ]:
MobileNet.save('final/MobileNet.h5')

In [ ]:
checkpoint_path = "NASNet/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

NASNet.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    shuffle=True,
    callbacks=[cp_callback]
)

In [ ]:
NASNet.save('final/NASNet.h5')